In [1]:
# Install dependencies
!pip uninstall -y tensorflow
!pip install transformers

Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 10.1MB/s 
     |████████████████████████████████| 1.1MB 30.4MB/s 
     |████████████████████████████████| 890kB 34.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=aa5a5bc45a01d6a36ded9f351e8d1e90df8c3d8d5d37f747a231924783c14572
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import json
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import transformers
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from transformers import BertModel, BertTokenizer

import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

In [4]:
class SentimentClassifier(nn.Module):
  """
  This class defines the model architecture which is simply a fully-connected
  layer on top of a pre-trained BERT model. 
  """

  def __init__(self, BERT_MODEL):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 2) # Binary Classifier

  def forward(self, ids, mask, token_type_ids):
    last_hidden_state, pooled_output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
    output = self.drop(pooled_output)
    return self.out(output)

# Inference on BERT-pair NLI-B

In [ ]:
class SentiHood:
  """
  This class tokenizes the input text using the pre-trained BERT tokenizer 
  (wordpiece) and returns the corresponding tensors.
  """
  
  def __init__(self, opinions_id, text, auxiliary_sentence, targets, tokenizer, max_len):
    self.opinions_id = opinions_id
    self.text = text
    self.auxiliary_sentence = auxiliary_sentence
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, item):
    opinions_id = self.opinions_id[item]
    text = str(self.text[item])
    auxiliary_sentence = str(self.auxiliary_sentence[item])
    targets = self.targets[item]

    text = text + ' ' + auxiliary_sentence

    inputs = self.tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = self.max_len,
        pad_to_max_length = True
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    return {
        "ids": torch.tensor(ids, dtype=torch.long),
        "mask": torch.tensor(mask, dtype=torch.long),
        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        "targets": torch.tensor(targets, dtype=torch.long),
        "opinions_id": torch.tensor(opinions_id, dtype=torch.long)
    }

In [ ]:
def infer_loop_function(data_loader, model, device):
  """
  This function performs the inference on testing sets and stores the predicted
  values.
  """

  model.eval()

  df_pred = pd.DataFrame({"id": [], "probability": [], "actual": []})

  ii = 0
  for bi, d in tqdm(enumerate(data_loader), total=len(data_loader), ncols=80):
    opinions_id = d["opinions_id"]
    ids = d["ids"]
    mask = d["mask"]
    token_type_ids = d["token_type_ids"]
    targets = d["targets"]

    opinions_id = opinions_id.to(device, dtype=torch.long)
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    targets = targets.to(device, dtype=torch.long)

    outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
    outputs = F.softmax(outputs, dim=-1)

    outputs = outputs.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    opinions_id = opinions_id.detach().cpu().numpy()

    # Storing the probability of "yes" as the matching score.
    for k in range(len(outputs)):
      df_pred.loc[ii] = [str(opinions_id[k]), outputs[k][1], targets[k]]
      ii += 1

  df_pred.to_csv('/content/drive/My Drive/SentiHood/Bert-pair/NLI-B/PredictedValues.csv', index=False)

In [ ]:
def run():
  """
  This function defines the necessary hyperparameters and models. It also 
  loads and tokenizes the testing dataset and execute the inference procedure.
  """

  TRAIN_MAX_LEN = 160
  TRAIN_BATCH_SIZE = 16
  BERT_MODEL = 'bert-base-uncased'

  testing_set_path = '/content/drive/My Drive/SentiHood/Bert-pair/NLI-B/Datasets/testing_set.csv'

  df_test = pd.read_csv(testing_set_path)
  df_test = df_test.reset_index(drop=True)

  tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL)

  test_dataset = SentiHood(
      opinions_id = df_test['id'].values,
      text = df_test['text'].values,
      auxiliary_sentence = df_test['auxiliary_sentence'],
      targets = df_test['sentiment'].values,
      tokenizer = tokenizer,
      max_len = TRAIN_MAX_LEN
  )
  print(f"Testing Set: {len(test_dataset)}")

  test_data_loader = torch.utils.data.DataLoader(
      test_dataset,
      batch_size = TRAIN_BATCH_SIZE,
      shuffle=False
  )

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print(f"Device: {device}")

  model = torch.load('/content/drive/My Drive/SentiHood/Bert-pair/NLI-B/Models/best.bin')
  infer_loop_function(data_loader=test_data_loader, model=model, device=device)
      
if __name__ == "__main__":
  run()

Testing Set: 67644
Device: cuda:0


100%|███████████████████████████████████████| 4228/4228 [15:48<00:00,  4.46it/s]


# Evaluation of BERT-pair NLI-B

In [5]:
"""
Select the sequence with highest matching score, to compute the sentiment.
"""

df_NLIB = pd.read_csv('/content/drive/My Drive/SentiHood/Bert-pair/NLI-B/PredictedValues.csv')
df_NLIM = pd.read_csv('/content/drive/My Drive/SentiHood/Bert-pair/NLI-M/PredictedValues.csv')

df = pd.DataFrame({"id": [], "predicted": []})

ii = 0
for k in tqdm(range(0, df_NLIB.shape[0]-3, 3), ncols=80):
  positive_probability = df_NLIB.iloc[k]['probability']
  negative_probability = df_NLIB.iloc[k+1]['probability']
  none_probability = df_NLIB.iloc[k+2]['probability']

  if positive_probability > negative_probability and positive_probability > none_probability:
    df.loc[ii] = [str(df_NLIB.iloc[k]['id']), 0]

  elif negative_probability > positive_probability and negative_probability > none_probability:
    df.loc[ii] = [str(df_NLIB.iloc[k]['id']), 1]

  else:
    df.loc[ii] = [str(df_NLIB.iloc[k]['id']), 2]
  
  ii += 1

df['predicted'] = df['predicted'].astype(int)
df['actual'] = df_NLIM['actual']

100%|████████████████████████████████████| 22547/22547 [01:28<00:00, 255.86it/s]


In [ ]:
def compute_sentiment_accuracy(df):
  """This function computes the sentiment classfication accuracy"""
  
  accuracy = df[df['predicted'] == df['actual']].shape[0]/df.shape[0] * 100
  return round(accuracy, 2)

print(f'Sentiment Accuracy of BERT-pair NLI-B = {compute_sentiment_accuracy(df)}%')

Sentiment Accuracy of BERT-pair NLI-B = 96.66%


In [ ]:
def compute_aspect_f1_score(df):
  """
  This function computest the macro F1 score of predicted aspects.
  0 => Represents that the aspect has not been detected.
  1 => Represents that the aspect has been detected.
  """
  
  df = df.replace([0, 1], 1).replace(2, 0)

  total_f1_score = 0
  total = 0
  
  for i in range(0, df.shape[0], 12):
    true_values = df.iloc[i:i+12]['predicted']
    predicted_values = df.iloc[i:i+12]['actual']

    total_f1_score += f1_score(true_values, predicted_values, average="macro")
    total += 1

  score = float(total_f1_score)/float(total)*100
  return round(score, 2)

print(f"Aspect F1 score: {compute_aspect_f1_score(df)}")

Aspect F1 score: 86.51


In [ ]:
def compute_aspect_accuracy(df):
  """
  This function computes the strict aspect accuracy.
  0 => Represents that the aspect has not been detected.
  1 => Represents that the aspect has been detected.
  """
  
  df = df.replace([0, 1], 1).replace(2, 0)

  count = 0
  total = 0

  for i in range(0, df.shape[0], 12):
    true_values = df.iloc[i:i+12]['predicted']
    predicted_values = df.iloc[i:i+12]['actual']

    if (true_values == predicted_values).all():
      count += 1
    total += 1

  accuracy = float(count)/float(total)*100
  return round(accuracy, 2)

print(f'Aspect Accuracy (strict) of BERT-pair NLI-B = {compute_aspect_accuracy(df)}%')

Aspect Accuracy (strict) of BERT-pair NLI-B = 68.28%


# Prediction Result Analysis

This section analyses the predicted results to find the aspects and sentiments that are most and least accurate.

*Note*: Utilizing the fact that first 1491x12 entries in the **above** constructed `df` are related to `location-1` and rest are related to `location-2`. 

In [6]:
"""
Computes the positive correct, positive total, negative correct, negative total, 
none correct, none total corresponding to all the aspects of LOCATION1.
"""

aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']
location1_aspects_result_analysis = {}

for i in range(12):
  location1_aspects_result_analysis[aspects[i]] = [[0 ,0], [0 ,0], [0 ,0]]

for i in tqdm(range(0, df['id'].unique().shape[0]*12-12, 12), ncols=80):
  for j in range(12):
    if df.loc[i+j]['actual'] == df.loc[i+j]['predicted']:
      location1_aspects_result_analysis[aspects[j]][int(df.loc[i+j]['actual'])][0] += 1
    
    location1_aspects_result_analysis[aspects[j]][int(df.loc[i+j]['actual'])][1] += 1

100%|██████████████████████████████████████| 1490/1490 [00:12<00:00, 115.93it/s]


In [7]:
df_location_aspect = pd.DataFrame({"location": [], "aspect": [], "positive correct": [], "positive total": [], "negative correct": [], "negative total": [], "none correct": [], "none total": [],})

ii = 0
for key in location1_aspects_result_analysis.keys():
  df_location_aspect.loc[ii] = ['LOCATION1', f"{key}", 
                                location1_aspects_result_analysis[key][0][0], 
                                location1_aspects_result_analysis[key][0][1], 
                                location1_aspects_result_analysis[key][1][0], 
                                location1_aspects_result_analysis[key][1][1], 
                                location1_aspects_result_analysis[key][2][0], 
                                location1_aspects_result_analysis[key][2][1]]
  ii += 1

In [8]:
"""
Computes the positive correct, positive total, negative correct, negative total, 
none correct, none total corresponding to all the aspects of LOCATION2.
"""

aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']
location2_aspects_result_analysis = {}

for i in range(12):
  location2_aspects_result_analysis[aspects[i]] = [[0 ,0], [0 ,0], [0 ,0]]

for i in tqdm(range(df['id'].unique().shape[0]*12, df.shape[0]-12, 12), ncols=80):
  for j in range(12):
    if df.loc[i+j]['actual'] == df.loc[i+j]['predicted']:
      location2_aspects_result_analysis[aspects[j]][int(df.loc[i+j]['actual'])][0] += 1
    
    location2_aspects_result_analysis[aspects[j]][int(df.loc[i+j]['actual'])][1] += 1

100%|████████████████████████████████████████| 387/387 [00:03<00:00, 117.93it/s]


In [9]:
for key in location2_aspects_result_analysis.keys():
  df_location_aspect.loc[ii] = ['LOCATION2', f"{key}", 
                                location2_aspects_result_analysis[key][0][0], 
                                location2_aspects_result_analysis[key][0][1], 
                                location2_aspects_result_analysis[key][1][0], 
                                location2_aspects_result_analysis[key][1][1], 
                                location2_aspects_result_analysis[key][2][0], 
                                location2_aspects_result_analysis[key][2][1]]
  ii += 1

In [10]:
df_location_aspect['positive percentage'] = round(df_location_aspect['positive correct']/df_location_aspect['positive total']*100, 2)
df_location_aspect['negative percentage'] = round(df_location_aspect['negative correct']/df_location_aspect['negative total']*100, 2)
df_location_aspect['none percentage'] = round(df_location_aspect['none correct']/df_location_aspect['none total']*100, 2)

df_location_aspect['total percentage'] = round((df_location_aspect['positive correct'] + df_location_aspect['negative correct'] + df_location_aspect['none correct'])/(df_location_aspect['positive total'] + df_location_aspect['negative total'] + df_location_aspect['none total'])*100, 2)

In [11]:
df_location_aspect

,location,aspect,positive correct,positive total,negative correct,negative total,none correct,none total,positive percentage,negative percentage,none percentage,total percentage
0,LOCATION1,dining,29.0,30.0,1.0,2.0,1453.0,1458.0,96.67,50.00,99.66,99.53
1,LOCATION1,general,272.0,359.0,81.0,113.0,928.0,1018.0,75.77,71.68,91.16,85.97
2,LOCATION1,green-nature,37.0,40.0,0.0,0.0,1433.0,1450.0,92.50,NaN,98.83,98.66
3,LOCATION1,live,48.0,63.0,15.0,23.0,1361.0,1404.0,76.19,65.22,96.94,95.57
4,LOCATION1,multicultural,31.0,39.0,1.0,3.0,1441.0,1448.0,79.49,33.33,99.52,98.86
5,LOCATION1,nightlife,57.0,62.0,1.0,2.0,1413.0,1426.0,91.94,50.00,99.09,98.72
6,LOCATION1,price,66.0,81.0,103.0,116.0,1268.0,1293.0,81.48,88.79,98.07,96.44
7,LOCATION1,quiet,7.0,14.0,12.0,15.0,1458.0,1461.0,50.00,80.00,99.79,99.13
8,LOCATION1,safety,54.0,61.0,52.0,66.0,1336.0,1363.0,88.52,78.79,98.02,96.78
9,LOCATION1,shopping,53.0,62.0,1.0,1.0,1421.0,1427.0,85.48,100.00,99.58,98.99


# Creating preds.jsonl

This section constructs the `preds.jsonl` file which contains model predictions and original annotations in the following json format.


```
{
  "opinions": [
    {
      "sentiment": "Positive",
      "aspect": "safety",
      "target_entity": "LOCATION1"
    }
  ],
  "id": 153,
  "text": " LOCATION1 is in Greater London and is a very safe place",
  "model_pred": [
    {
      "sentiment": ...,
      "aspect": ...,
      "target_entity":...
    },...
  ]
}
```

In [ ]:
with open('/content/drive/My Drive/SentiHood/SentiHood Dataset/sentihood-test.json', 'r') as fp:
  testing_set = json.load(fp)

In [ ]:
labels_to_sentiment_dict = {
    0: 'Positive',
    1: 'Negative',
    2: 'None'
}

In [ ]:
BERT_MODEL = 'bert-base-uncased'
MAX_LEN = 160
locations = ['LOCATION1', 'LOCATION2']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']

tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

model = torch.load('/content/drive/My Drive/SentiHood/Bert-pair/NLI-B/Models/best.bin')

for each_example in tqdm(testing_set, ncols=80):
  id = each_example['id']
  text = each_example['text'].strip()

  each_example['model_pred'] = []

  count_location = 1
  for location in locations:
    if location in text:
       # If "location" is present in the text, then utilize the trained model
      # to predict the aspects and their corresponding sentiment of the text.

      text = text.replace(location, 'location - ' + str(count_location))
      
      for aspect in aspects:
        predicted_probabilities = []
        aspect_sentiment = None

        for polarity in ['Positive', 'Negative', 'None']:
          auxiliary_sentence = f"the polarity of the aspect {aspect} of location - {str(count_location)} is {polarity}."
          
          combined_text = text + ' ' + auxiliary_sentence
          
          inputs = tokenizer.encode_plus(
              combined_text,
              add_special_tokens = True,
              max_length = MAX_LEN,
              pad_to_max_length = True
          )
          ids = torch.tensor(inputs["input_ids"], dtype=torch.long).unsqueeze(0)
          mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).unsqueeze(0)
          token_type_ids = torch.tensor(inputs["token_type_ids"], dtype=torch.long).unsqueeze(0)

          ids = ids.to(device, dtype=torch.long)
          mask = mask.to(device, dtype=torch.long)
          token_type_ids = token_type_ids.to(device, dtype=torch.long)

          outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
          outputs = F.softmax(outputs, dim=-1)

          outputs = outputs.detach().cpu().numpy()

          predicted_probabilities.append(outputs[0][1])

        # Utilizing the matching score to compute the sentiment.
        if predicted_probabilities[0] > predicted_probabilities[1] and predicted_probabilities[0] > predicted_probabilities[2]:
          aspect_sentiment = 'Positive'
        elif predicted_probabilities[1] > predicted_probabilities[0] and predicted_probabilities[1] > predicted_probabilities[2]:
          aspect_sentiment = 'Negative'
        else:
          aspect_sentiment = None

        # If predicted sentiment is not None, then add it to the preds.jsonl.
        
        if aspect_sentiment:
          result = {
              "sentiment": aspect_sentiment,
              "aspect": aspect,
              "target_entity": location
          }
          each_example['model_pred'].append(result)
      
    count_location += 1


Device: cuda:0


100%|███████████████████████████████████████| 1491/1491 [41:16<00:00,  1.66s/it]


In [ ]:
with open('/content/drive/My Drive/SentiHood/Bert-pair/NLI-B/preds.jsonl', mode='w', encoding='utf-8') as fp:
  for each in testing_set:
    json_record = json.dumps(each, ensure_ascii=False)
    fp.write(json_record + '\n')